In [1]:
import numpy as np
import pandas as pd
import nltk
import time
nltk.download('stopwords')

from numpy import random as rd
from scipy.special import gammaln
from nltk.corpus import stopwords
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sara\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Function definitions

## File related

In [2]:
def find_n_lines(filename, encoding, n_tokens):
    """
    We should train on a subset of the corpus, with 100 0000 tokens. Find how many lines this corresponds to.
    """
    
    line_counter = 0
    token_counter = 0

    with open(filename, encoding = encoding) as f:
        for line in f:

            line_counter += 1
            tokens = line.lower().split()

            for token in tokens:
                token_counter += 1

            if token_counter > n_tokens:
                break

    return line_counter


def count_word_frequencies(filename, encoding, n_lines, ignore_list):
    
    freqs = Counter()
    with open(filename, encoding = encoding) as f:
        for i, line in enumerate(f):
            
            tokens = line.lower().split()
            for token in tokens:
                if token not in ignore_list:
                    freqs[token] += 1
                
            if i == n_lines:
                break
                
    return freqs

In [3]:
def list_of_stopwords():
    # Ignore all stopwords in the text!
    ignore_words = stopwords.words('english')
    also_ignore = [",", ".", '"', "(", ")", "-", "'", "!", "?", ":", ";", "/", "n't", "'s", "'m"]

    for item in also_ignore:
        ignore_words.append(item)
        
    return ignore_words

## Batch related

In [164]:
def create_integer_vocabulary(word_freqs, max_voc_size):
    
    """ 
    Create vocabulary where common words are matched to integers. 
    """
    
    word_list = []

    if len(word_freqs.most_common()) > max_voc_size:
        vocab = word_freqs.most_common()[0:max_voc_size]

    else:
        vocab = word_freqs.most_common()

    for i in range(len(vocab)):
        word_list.append(vocab[i][0])

    # Get pairs of elements    
    tmp = zip(word_list, range(1,max_voc_size+1))
    # Make pairs into a dictionary
    vocab = dict(tmp)

    # Create default dictionary - returns 0 if an undefined key is called
    vocab2 = defaultdict(int)
    vocab2.update(vocab)
    
    return vocab2

def find_batch_dimensions(batch_size, filename, ENCODING):
    """
    Find the length of the longest line in each batch.
    """
    
    counter = 0      # will end up being the number of lines in the document
    len_lines = []   # will contain maximum length of a line in each batch
    tmp_lines = []
    
    with open(filename, encoding=ENCODING) as f:
        for line in f:
            counter+=1
            tokens = line.lower().split()
            tmp_lines.append(len(tokens))

            if (counter % batch_size == 0):
                len_lines.append(max(tmp_lines))
                tmp_lines = []
                
        #This takes care of the last batch if number of lines is not an exact multiple of batch_size
        if (counter % batch_size != 0): 
            len_lines.append(max(tmp_lines)) # if at end of the file
            
    return counter, len_lines
    

def create_batches(batch_size, vocabulary, filename, ENCODING):
    """
    Splits the file into batches of a specified size, and transforms common words to integers.
    The batches are outputted in a numpy array padded with zeros. Words not in the vocabulary are set to -1.
    """
    
    counter, len_lines = find_batch_dimensions(batch_size, filename, ENCODING)
    
    with open(filename, encoding=ENCODING) as f:
        batches=[]
        batch_counter=0
        line_counter=0

        for line in f:
            #This creates a temporary array each time we start a new batch
            if line_counter % batch_size == 0:
                tmp_array=np.zeros(shape=(batch_size,len_lines[batch_counter])) #fill this temporary array

            tokens = line.lower().split()
            line_as_int = list(map(vocabulary.get, tokens))
            line_as_int = [-1 if x is None else x for x in line_as_int] # set None values to -1

            tmp_array[line_counter % batch_size,0:(len(line_as_int))]=line_as_int

            line_counter+=1 #when we done
            if line_counter % batch_size ==0:
                batches.append(tmp_array)
                batch_counter+=1

        # again this takes care of the final batch if number of lines is not multiple of batch_size
        if line_counter % batch_size != 0:
            tmp_array=tmp_array[0:(line_counter % batch_size),:]
            batches.append(tmp_array)
        
    return(counter, batches)

def get_matrix(filename, encoding, n_tokens, ignore_words):
    # Find how many lines we need to read to get the desired number of tokens
    n_docs = find_n_lines(filename, encoding, n_tokens)

    # Count word frequencies in this subset of the file
    word_frequencies = count_word_frequencies(filename, encoding, n_docs, ignore_words)

    # Create an integer vocabulary. Don't remove any words from the vocabulary.
    voc_size = len(word_frequencies)
    vocabulary = create_integer_vocabulary(word_frequencies, voc_size)

    # Turn the document into batches
    lines, batches = create_batches(batch_size=n_docs, vocabulary = vocabulary, filename = filename, ENCODING = encoding)

    # Save only the first batch - this is what we'll analyse
    matrix = batches[0].astype(int)
    return n_docs, matrix, vocabulary, voc_size

## LDA Related

In [5]:
def initialise_everything(n_docs, n_topics, voc_size, int_matrix):
    """
    Initialise the things we need for LDA.
    """

    # Number of times that we observe topic z in document d
    ndz = np.zeros((n_docs, n_topics))

    # Number of times that we observe word w in topic z
    nzw = np.zeros((n_topics, voc_size))

    # Counters for documents and topics
    nd = np.zeros(n_docs)
    nz = np.zeros(n_topics)

    # Create dictionary of topics
    topics = {}

    # iterate over documents 
    for d in range(n_docs):

        # i is the index of the word in the document
        # w is the numerical representation of the word
        for i, w in enumerate(int_matrix[d]):

            # Initialise with a random topic
            z = rd.randint(n_topics)
            topics[(d,i)] = z

            # Increase counters
            ndz[d, z] += 1
            nzw[z, w] += 1

            nd[d] += 1
            nz[z] += 1

    return topics, ndz, nzw, nd, nz

In [6]:
def cond_topic_prob(ndz, nzw, nz, nd, w, d, alpha, beta, n_topics):
    """
    Conditional probability of topics. 
    """

    left = (nzw[:,w] + beta) / (nz + beta * voc_size)
    right = (ndz[d,:] + alpha) / (nd[d] + alpha * n_topics)

    p_z = left * right
    p_z /= np.sum(p_z)
    
    return p_z

def log_multinomial_beta(alpha, K=None):

    if K is None:
        # alpha is assumed to be a vector
        return np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))
    else:
        # alpha is assumed to be a scalar
        return K * gammaln(alpha) - gammaln(K*alpha)

# This should increase as training progresses, show it every few training iterations (?)
def loglikelihood(n_topics, voc_size, alpha, beta, nzw, ndz):
    likelihood = 0
    
    for z in range(n_topics):
        likelihood += log_multinomial_beta(nzw[z,:] + beta)
        likelihood -= log_multinomial_beta(beta, voc_size)
        
    for d in range(n_docs):
        likelihood += log_multinomial_beta(ndz[d,:] + alpha)
        likelihood -= log_multinomial_beta(alpha, n_topics)
        
    return likelihood

In [7]:
def LDA_Gibbs_Sampler(matrix, voc_size, n_docs, n_topics, max_iterations, alpha, beta):

    start_time = time.time()
    topics, ndz, nzw, nd, nz = initialise_everything(n_docs, n_topics, voc_size, matrix)

    for i in range(max_iterations):
        for d in range(n_docs):
             for j, w in enumerate(matrix[d]):

                z = topics[(d, j)]
                ndz[d, z] -= 1
                nzw[z, w] -= 1
                nd[d] -= 1
                nz[z] -= 1

                p_z = cond_topic_prob(ndz, nzw, nz, nd, w, d, alpha, beta, n_topics)
                z = rd.multinomial(1, p_z).argmax()

                ndz[d,z] += 1
                nzw[z,w] += 1
                nd[d] += 1
                nz[z] += 1
                topics[(d, j)] = z

        print("Iteration", i)
        print("Likelihood", loglikelihood(n_topics, voc_size, alpha, beta, nzw, ndz))

    elapsed_time = time.time() - start_time
    print("Elapsed time: ", elapsed_time)
    
    return nzw

In [19]:
def show_words_by_topic(word_topic_prob, vocabulary, typical_len):
    
    n_topics = word_topic_prob.shape[0]
    typical_words = []

    for i in range(n_topics):
        arr = word_topic_prob[i,:]
        typical_ints = arr.argsort()[-typical_len-2:-2][::-1]   # there's some funny business with the last word in vocab
        #print(typical_ints)

        for search_int in typical_ints:
            if search_int in [0, -1]:
                typical_words.append("")
            else:
                for k, v in vocabulary.items(): 
                    if v == search_int:
                        typical_words.append(k)
                        break

    # Print the most common words in each topic
    typical_words = np.reshape(typical_words, [n_topics, -1])
    print(typical_words)


# 1: Write your own code for doing Gibbs sampling for LDA

The initial results from this algorithm were just nonsense (all topics mostly consisted of stopwords and grammatical symbols). This is why stopwords and grammatical symbols have been ignored in the vocabulary. 

The data matrix uses zero padding, which due to an oversight was not taken into account in this implementation. Thus there is an issue with the conditional probabilites; the number of words ends up being the same for each document and also, the counts for different topics is biased. However these issues appear to not have affected the end result too much, as reasonable looking results are obtained. 

Collapsed Gibbs sampling for LDA has been implemented, and the algorithm is tested on a subset of the Amazon book review corpus consisting of 80 000 tokens. The algorithm runs for 75 iterations and the log-likelihood is outputted at the end of each iteration. We tried using 10 and 50 number of topics, both with $\alpha = \beta = 0.1$ and $\alpha = \beta = 0.01$. Finally, the 10 most common words for each topic are printed.

In [9]:
filename = "books.txt" 
encoding = "ISO-8859-1"
n_tokens = 8*10**4

ignore_words = list_of_stopwords()

n_docs, matrix, vocabulary, voc_size = get_matrix(filename, encoding, n_tokens, ignore_words)

In [10]:
typical_len = 10
max_iterations = 75

### 10 Topics, $\alpha = \beta = 0.1$

In [11]:
n_topics = 10

word_topic_prob_01_10 = LDA_Gibbs_Sampler(matrix, voc_size, n_docs, n_topics, \
                                          max_iterations, alpha = 0.1, beta = 0.1)

Iteration 0
Likelihood -2351002.5387130915
Iteration 1
Likelihood -2335354.9581859065
Iteration 2
Likelihood -2323639.282311263
Iteration 3
Likelihood -2311577.561338965
Iteration 4
Likelihood -2296861.8465751046
Iteration 5
Likelihood -2276834.275514169
Iteration 6
Likelihood -2243067.8784760805
Iteration 7
Likelihood -2199891.9970693695
Iteration 8
Likelihood -2156739.2925651036
Iteration 9
Likelihood -2123050.5709214304
Iteration 10
Likelihood -2098328.7566538285
Iteration 11
Likelihood -2079706.214526914
Iteration 12
Likelihood -2065564.3258698783
Iteration 13
Likelihood -2052815.0407773168
Iteration 14
Likelihood -2041726.5628088245
Iteration 15
Likelihood -2028867.4095321808
Iteration 16
Likelihood -2017191.9350128223
Iteration 17
Likelihood -2005201.353564023
Iteration 18
Likelihood -1994140.9965495174
Iteration 19
Likelihood -1982265.8389743764
Iteration 20
Likelihood -1972151.617760897
Iteration 21
Likelihood -1961846.7944976867
Iteration 22
Likelihood -1952392.4721891806
Iter

In [20]:
show_words_by_topic(word_topic_prob_01_10, vocabulary, typical_len)

[['book' 'read' 'would' 'well' 'good' 'know' 'us' 'two' 'think' 'highly']
 ['read' 'one' 'books' 'people' 'world' 'man' 'works' 'time' 'new'
  'women']
 ['book' 'like' 'reading' 'would' 'new' 'much' 'lot' 'school' 'long'
  'ever']
 ['book' 'books' 'one' 'many' 'history' 'way' 'reading' 'life' 'people'
  'work']
 ['great' 'story' 'like' 'many' 'read' 'world' 'see' 'also' 'novel'
  'john']
 ['one' 'much' 'would' 'also' 'even' 'like' 'sam' 'get' 'time' 'could']
 ['great' 'reader' 'writing' 'love' 'must' 'different' 'information'
  'give' 'patterns' 'could']
 ['book' 'recommend' 'read' 'anyone' 'great' 'story' 'going'
  'interesting' 'information' 'would']
 ['book' 'one' 'good' 'first' 'really' 'best' 'time' 'find' 'work' 'get']
 ['life' 'war' 'new' 'family' 'us' '' 'without' 'theory' 'power'
  'american']]


Of the above topics, three appear to make sense:

- __"People" topic__: ['read' 'one' 'books' 'people' 'world' 'man' 'works' 'time' 'new'
  'women']
  
- __"Positive" topic__:  ['book' 'recommend' 'read' 'anyone' 'great' 'story' 'going'
  'interesting' 'information' 'would']
  
- __"Serious" topic__: ['life' 'war' 'new' 'family' 'us' '' 'without' 'theory' 'power'
  'american']

### 50 Topics, $\alpha = \beta = 0.1$

In [21]:
n_topics = 50

word_topic_prob_01_50 = LDA_Gibbs_Sampler(matrix, voc_size, n_docs, n_topics, \
                                          max_iterations, alpha = 0.1, beta = 0.1)

Iteration 0
Likelihood -3658563.936163828
Iteration 1
Likelihood -3608550.923346697
Iteration 2
Likelihood -3558537.080710233
Iteration 3
Likelihood -3508411.4883670104
Iteration 4
Likelihood -3449413.682229062
Iteration 5
Likelihood -3379673.1660486497
Iteration 6
Likelihood -3301858.0268891277
Iteration 7
Likelihood -3230342.4180519395
Iteration 8
Likelihood -3169399.4883624515
Iteration 9
Likelihood -3119861.807302928
Iteration 10
Likelihood -3077177.2423457718
Iteration 11
Likelihood -3035792.7022777093
Iteration 12
Likelihood -2998331.9405336394
Iteration 13
Likelihood -2963537.9445924326
Iteration 14
Likelihood -2929729.842392472
Iteration 15
Likelihood -2893592.647204069
Iteration 16
Likelihood -2859870.2313416135
Iteration 17
Likelihood -2825669.469561156
Iteration 18
Likelihood -2787576.176527688
Iteration 19
Likelihood -2750477.565235863
Iteration 20
Likelihood -2710736.847433333
Iteration 21
Likelihood -2672623.207860701
Iteration 22
Likelihood -2631332.23425144
Iteration 23

In [22]:
show_words_by_topic(word_topic_prob_01_50, vocabulary, typical_len)

[['monster' 'born' 'horrific' 'killed' 'fell' 'girls' 'memories' 'amelia'
  'telling' 'actual']
 ['cooke' 'biography' 'events' 'official' 'version' 'wolff' 'known'
  'greene' 'car' 'death']
 ['edition' 'thorough' 'girlfriend' 'nature' 'exactly' 'medieval' 'vast'
  'eugenics' 'exhaustive' 'analyzed']
 ['soul' 'human' 'xxiii' 'later' 'thomas' 'care' 'ladder' 'window'
  'monastic' 'grow']
 ['book' 'well' 'first' 'characters' 'good' 'story' 'read' 'writing'
  'enjoyed' 'amazing']
 ['politics' 'lbj' 'loved' 'mets' 'scientific' 'myron' 'bolitar' 'kathy'
  'years' 'career']
 ['one' 'like' 'would' 'also' 'read' 'many' 'people' 'life' 'much' 'time']
 ['energy' 'michelle' 'vital' 'mama' 'vampirism' 'age' 'peter' 'battle'
  'explosive' 'hearing']
 ['arguments' 'department' 'grisham' 'rhetorician' 'thousands' 'original'
  'creator' 'zen' 'motorcycle' 'relationship']
 ['reporter' 'reviewer' '...' 'universe' 'fox' 'mop' 'diverting'
  'plodding' 'time' 'background']
 ['reading' 'would' 'without' 'war

Of the above, 12 topics appear to make sense:

- __"Horror" topic__: ['monster' 'born' 'horrific' 'killed' 'fell' 'girls' 'memories' 'amelia'
  'telling' 'actual']
  
- __"Positive" topic__: ['book' 'well' 'first' 'characters' 'good' 'story' 'read' 'writing'
  'enjoyed' 'amazing']
  
- __"Myron Bolitar" topic__: ['politics' 'lbj' 'loved' 'mets' 'scientific' 'myron' 'bolitar' 'kathy'
  'years' 'career'] - LBJ = Lyndon B Johnson, 36th president of the USA
  
 
- __"Science" topic__: ['adso' 'rose' 'word' 'st.' 'modern' 'scientist' 'thus' 'knowledge'
  'bernard' 'science']
  
- __"Italian restaurant" topic__: ['service' 'served' 'evening' 'wine' 'euro' 'italian' 'venice' 'roasted'
  'green' 'food']
  
- __"Alien abduction" topic__: ['abduction' 'field' 'hallucinations' 'consciousness' 'darwin'
  'alternative' 'evolutionary' 'loves' 'conclusion' 'captures']
 
 
- __"Hurricane Katrina" topic__: ['style' 'orleans' 'corps' 'needs' 'south' 'american' 'lies' 'river'
  'katrina' 'new']
  
- __"Terrorism" topic__:  ['memorable' 'surface' 'terrorism' 'laden' 'ali' 'terrorist' 'handbook'
  'note' 'johnson' 'parallels']
  
- __"ID check" topic__:  ['drivers' 'humans' 'step' 'license' 'id' 'control' 'wisdom' 'act'
  'passport' 'helped']
  
  
- __"Companies" topic__:  ['companies' 'mountains' 'created' 'happy' 'jobs' 'wage' 'development'
  'issue' 'former' 'highest']
 
- __"Terry Pratchett__" topic: ['series' 'discworld' 'equal' 'granny' 'magic' 'killer' 'notre' 'dame'
  'immigrant' 'witches']
  
- __"Evolution" topic__: ['students' 'text' 'alone' 'common' 'design' 'evolution' 'opinion' 'pay'
  'darwin' 'show']

### 10 Topics, $\alpha = \beta = 0.01$

In [23]:
n_topics = 10

word_topic_prob_001_10 = LDA_Gibbs_Sampler(matrix, voc_size, n_docs, n_topics, max_iterations, alpha = 0.01, beta = 0.01)

Iteration 0
Likelihood -2343457.870990367
Iteration 1
Likelihood -2320749.8681018814
Iteration 2
Likelihood -2304569.705220574
Iteration 3
Likelihood -2290597.510808511
Iteration 4
Likelihood -2276621.381299429
Iteration 5
Likelihood -2260341.027312455
Iteration 6
Likelihood -2235869.9136667443
Iteration 7
Likelihood -2202960.071286286
Iteration 8
Likelihood -2165050.2412849767
Iteration 9
Likelihood -2129827.7747944077
Iteration 10
Likelihood -2101752.4814116503
Iteration 11
Likelihood -2080085.3136327907
Iteration 12
Likelihood -2063487.1144998441
Iteration 13
Likelihood -2045536.251464159
Iteration 14
Likelihood -2031064.2508593616
Iteration 15
Likelihood -2016179.4829783826
Iteration 16
Likelihood -2000026.3275540767
Iteration 17
Likelihood -1985799.924806189
Iteration 18
Likelihood -1974548.4329042924
Iteration 19
Likelihood -1960551.3750523948
Iteration 20
Likelihood -1949357.0448735512
Iteration 21
Likelihood -1937563.1301793077
Iteration 22
Likelihood -1923682.851831316
Iterati

In [24]:
show_words_by_topic(word_topic_prob_001_10, vocabulary, typical_len)

[['book' 'good' 'best' 'author' 'new' 'reader' 'every' 'well' 'like'
  'excellent']
 ['book' 'would' 'much' 'even' 'work' 'history' 'two' 'reading' 'like'
  'novel']
 ['work' 'love' 'first' 'wonderful' 'easy' 'real' 'lot' 'use' 'amazing'
  'patterns']
 ['one' 'book' 'characters' 'time' 'world' 'best' 'like' 'stories'
  'could' 'although']
 ['book' 'good' 'pages' 'interesting' 'woman' 'high' 'recommend' 'long'
  'still' 'stories']
 ['book' 'read' 'great' 'many' 'must' 'recommend' 'better' 'also'
  'series' 'students']
 ['books' 'time' 'readers' 'fiction' 'children' 'reader' 'need' 'says'
  'novels' 'american']
 ['book' 'written' 'information' 'world' 'worth' 'highly' 'novel'
  'anyone' 'page' 'complete']
 ['book' 'life' 'people' 'us' 'story' 'also' 'one' 'read' 'first' 'many']
 ['book' 'great' 'would' 'reading' 'years' 'way' 'well' 'writing' 'works'
  'time']]


Of the above topics, four appear to make sense:

- __"Positive" topic__: ['book' 'good' 'best' 'author' 'new' 'reader' 'every' 'well' 'like'
  'excellent']
  
- __"Positive" topic 2__: ['work' 'love' 'first' 'wonderful' 'easy' 'real' 'lot' 'use' 'amazing'
  'patterns']
  
- __"Recommending other books in series" topic__:  ['book' 'read' 'great' 'many' 'must' 'recommend' 'better' 'also'
  'series' 'students']
  
- __"Children's fiction" topic__: ['books' 'time' 'readers' 'fiction' 'children' 'reader' 'need' 'says'
  'novels' 'american']

Notice: two similar topics. Also, there are no negative topics! Maybe that is from the corpus, or maybe because we don't consider negations. 

Compare the loglikelihood between models

### 50 Topics, $\alpha = \beta = 0.01$

In [25]:
n_topics = 50

word_topic_prob_001_50 = LDA_Gibbs_Sampler(matrix, voc_size, n_docs, n_topics, max_iterations, alpha = 0.01, beta = 0.01)

Iteration 0
Likelihood -3575222.7566558477
Iteration 1
Likelihood -3511858.157047361
Iteration 2
Likelihood -3453380.977278283
Iteration 3
Likelihood -3396604.8616934363
Iteration 4
Likelihood -3330508.159939607
Iteration 5
Likelihood -3249983.8052112306
Iteration 6
Likelihood -3174941.4953452684
Iteration 7
Likelihood -3108042.3533887807
Iteration 8
Likelihood -3052397.1337047815
Iteration 9
Likelihood -3003682.1830039476
Iteration 10
Likelihood -2959216.6975951935
Iteration 11
Likelihood -2920272.050352145
Iteration 12
Likelihood -2881800.5491413083
Iteration 13
Likelihood -2848358.745659724
Iteration 14
Likelihood -2816436.5785797634
Iteration 15
Likelihood -2786236.584752912
Iteration 16
Likelihood -2757118.728225909
Iteration 17
Likelihood -2726209.2886329475
Iteration 18
Likelihood -2695776.095584607
Iteration 19
Likelihood -2666513.283767589
Iteration 20
Likelihood -2639867.317621984
Iteration 21
Likelihood -2615079.749422195
Iteration 22
Likelihood -2591513.5442678286
Iteration

In [26]:
show_words_by_topic(word_topic_prob_001_50, vocabulary, typical_len)

[['great' 'much' 'way' 'must' 'long' 'series' 'esk' 'put' 'want' 'could']
 ['story' 'books' 'read' 'first' 'daughter' 'us' 'period' 'however'
  'times' 'writer']
 ['one' 'also' 'give' 'first' 'review' 'knowledge' 'need' 'business'
  'serious' 'bring']
 ['history' 'students' 'c' 'second' 'years' 'journey' 'student' 'ago'
  'long' 'fantastic']
 ['book' 'war' 'author' 'getting' 'later' 'third' 'particularly' 'though'
  'example' 'library']
 ['things' 'helped' 'days' 'practical' '&' 'picture' 'understand' 'liked'
  'plan' 'lessons']
 ['one' 'writing' 'could' 'written' 'women' 'recommended' 'recommend'
  'patterns' 'school' 'would']
 ['excellent' 'highly' 'different' 'anyone' 'tale' 'works' 'kind' 'angel'
  'classic' 'good']
 ['people' 'read' 'really' 'experience' 'rest' 'shows' 'society'
  'written' 'myth' 'engineers']
 ['gibbon' 'work' 'could' 'mother' 'going' 'reading' 'fact' 'time' 'new'
  'perhaps']
 ['children' 'life' 'human' 'freddie' 'experience' 'helga' 'soul' 'care'
  'spiritual' 

- __"Lesson planning" topic__: ['things' 'helped' 'days' 'practical' '&' 'picture' 'understand' 'liked'
  'plan' 'lessons']
  
- __"Positive" topic__:  ['great' 'reading' 'best' 'say' 'detail' 'also' 'something' 'works'
  'enjoyed' 'writes']

- __"Age" topic__:  ['named' 'old' 'research' 'age' 'studies' 'boy' 'page' 'pearl'
  'including' 'man']

the words "book" and "read" are very common here - seems like this one didn't learn as much. 

### Save results as csv

In [33]:
np.savetxt("nzw_01_10.csv", word_topic_prob_01_10, delimiter=",")
np.savetxt("nzw_01_50.csv", word_topic_prob_01_50, delimiter=",")
np.savetxt("nzw_001_10.csv", word_topic_prob_001_10, delimiter=",")
np.savetxt("nzw_001_50.csv", word_topic_prob_001_50, delimiter=",")

# 2: Write your own code for doing Gibbs samling on Bigram LDA

## Function definitions

In [658]:
def count_bigram_topics(filename, encoding, n_lines, vocabulary, ignore_list, topics):
        
    string_dict = defaultdict(Counter)
    int_dict = defaultdict(Counter)
    
    with open(filename, encoding = encoding) as f:
        for d, line in enumerate(f):
            
            if d == n_lines:
                break
            
            tokens = line.lower().split()
  
            for i, (t1, t2) in enumerate(zip(tokens, tokens[1:])):
                
                if t2 not in ignore_list:
                    z = topics[(d, i+1)]

                    string_dict[(t1, t2)][z] += 1

                    int1 = vocabulary[t1]
                    int2 = vocabulary[t2]
                    int_dict[(int1, int2)][z] += 1
            
    return string_dict, int_dict

In [386]:
def cond_prob_bigram(d, ndz, nzw, nz, nd, w, w_old, alpha, beta, n_topics, bigrams_num):
    p_z = np.zeros(n_topics)

    for k in range(n_topics):
        left = ( bigrams_num[(w_old, w)][k] + beta * np.sum(nzw[:,w]) )/( nzw[k, w_old + 1] + beta )
        right = ( ndz[d,k] + alpha * nz[k] )/( nd[d] + alpha )

        p_z[k] = left*right

    p_z /= sum(p_z)
    return p_z

In [665]:
def initialise_for_bigram(matrix, n_docs, n_topics, voc_size):
    
    ndz = np.zeros((n_docs, n_topics))
    nzw = np.zeros((n_topics, voc_size))
    nd = np.zeros(n_docs)
    nz = np.zeros(n_topics)
    topics = {}

    for d, line in enumerate(matrix):
        for i, w in enumerate(line):

            if w not in [-1, 0]:

                # Initialise with a random topic
                z = rd.randint(n_topics)
                topics[(d,i)] = z
                ndz[d, z] += 1
                nzw[z, w] += 1
                nd[d] += 1
                nz[z] += 1
                
    return ndz, nzw, nd, nz, topics

In [668]:
def run_bigram_LDA(ndz, nzw, nd, nz, topics, matrix, max_iterations, alpha, beta, n_topics, bigrams_num):

    something_went_wrong = 0

    start_time = time.time()

    for i in range(max_iterations):
        for d, line in enumerate(matrix):
            for j, w in enumerate(line):
                if w not in [-1, 0]:

                    z = topics[(d, j)]
                    ndz[d, z] -= 1
                    nzw[z, w] -= 1
                    nd[d] -= 1
                    nz[z] -= 1

                    # preceding word in the bigram   
                    w_prev = matrix[d, j-1]

                    # If the word only appears once in corpus
                    if np.sum(nzw[:,w]) == 0:
                        z = rd.randint(n_topics)      

                    elif bigrams_num[(w_prev, w)][z] == 0:

                        something_went_wrong += 1
                        z = rd.randint(n_topics)      

                    else:

                        bigrams_num[(w_prev, w)][z] -= 1
                        p_z = cond_prob_bigram(d, ndz, nzw, nz, nd, w, w_prev, alpha, beta, n_topics, bigrams_num)
                        z = rd.multinomial(1, p_z).argmax()

                    ndz[d,z] += 1
                    nzw[z,w] += 1
                    nd[d] += 1
                    nz[z] += 1
                    topics[(d, j)] = z

                    bigrams_num[(w_prev, w)][z] += 1

        print("Iteration", i)

    elapsed_time = time.time() - start_time
    print("Elapsed time: ", elapsed_time)

    return nzw, something_went_wrong

## Code

In [659]:
with open(filename, encoding = encoding) as f:
    for line in f:
        print(line)
        break

it took me two days to read this book because once i started i could n't put it down . mr.magnus leads us through various exercises to show us how we too can have an astral projection . the exercises are fun and interesting . he also adds entries from his journal that makes the reader feel that they too can achieve the ability to enter the astral realm . this book is easy to read unlike many other books on the subject which make the reader feel bogged down with specificities which soon become boring . a beautifully written , enthusiastic book about a subject that is surrounded in mystery 



In [671]:
# This step is exactly the same as in part 1

filename = "books.txt" 
encoding = "ISO-8859-1"
n_tokens = 5*10**5

ignore_words = list_of_stopwords()
n_docs, matrix, vocabulary, voc_size = get_matrix(filename, encoding, n_tokens, ignore_words)

In [672]:
max_iterations = 100
beta = 0.1
alpha = 0.1

### N_topics = 5

In [673]:
n_topics = 5

ndz, nzw, nd, nz, topics = initialise_for_bigram(matrix, n_docs, n_topics, voc_size)
bigrams, bigrams_num = count_bigram_topics(filename, encoding, n_docs, vocabulary, ignore_words, topics)
probs_5, error_counter = run_bigram_LDA(ndz, nzw, nd, nz, topics, matrix, max_iterations, alpha, beta, n_topics, bigrams_num)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [674]:
show_words_by_topic(probs_5, vocabulary, typical_len = 10)

[['like' 'one' 'books' 'reading' 'would' 'people' 'time' 'story'
  'recommend' 'us']
 ['read' 'like' 'story' 'would' 'books' 'years' 'time' 'reading' 'life'
  'people']
 ['read' 'great' 'would' 'good' 'many' 'first' 'well' 'also' 'story'
  'like']
 ['like' 'one' 'books' 'would' 'time' 'years' 'reading' 'life' 'written'
  'story']
 ['like' 'one' 'reading' 'recommend' 'books' 'time' 'would' 'story'
  'history' 'life']]


None of these make sense?

[['like' 'one' 'books' 'reading' 'would' 'people' 'time' 'story'
  'recommend' 'us']
  
 ['read' 'like' 'story' 'would' 'books' 'years' 'time' 'reading' 'life'
  'people']
  
 ['read' 'great' 'would' 'good' 'many' 'first' 'well' 'also' 'story'
  'like']
  
 ['like' 'one' 'books' 'would' 'time' 'years' 'reading' 'life' 'written'
  'story']
  
 ['like' 'one' 'reading' 'recommend' 'books' 'time' 'would' 'story'
  'history' 'life']]

### N_topics = 10

In [675]:
n_topics = 10

ndz, nzw, nd, nz, topics = initialise_for_bigram(matrix, n_docs, n_topics, voc_size)
bigrams, bigrams_num = count_bigram_topics(filename, encoding, n_docs, vocabulary, ignore_words, topics)
probs_10, error_counter = run_bigram_LDA(ndz, nzw, nd, nz, topics, matrix, max_iterations, alpha, beta, n_topics, bigrams_num)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [676]:
show_words_by_topic(probs_10, vocabulary, typical_len = 10)

[['like' 'would' 'one' 'years' 'reading' 'books' 'time' 'us' 'story'
  'people']
 ['read' 'one' 'books' 'would' 'time' 'years' 'story' 'reading' 'new'
  'us']
 ['like' 'one' 'reading' 'books' 'time' 'story' 'us' 'would' 'years'
  'life']
 ['read' 'century' 'one' 'books' 'people' 'reading' 'time' 'would' 'life'
  'story']
 ['like' 'books' 'one' 'reading' 'would' 'story' 'life' 'years' 'time'
  'us']
 ['read' 'great' 'good' 'would' 'many' 'also' 'well' 'story' 'first'
  'much']
 ['one' 'like' 'recommend' 'books' 'years' 'would' 'reading' 'people'
  'life' 'us']
 ['like' 'one' 'books' 'recommend' 'reading' 'would' 'time' 'life' 'work'
  'people']
 ['one' 'like' 'would' 'reading' 'years' 'books' 'story' 'time' 'people'
  'history']
 ['like' 'one' 'would' 'books' 'reading' 'story' 'people' 'time' 'years'
  'way']]


None of these really make sense?

[['like' 'would' 'one' 'years' 'reading' 'books' 'time' 'us' 'story'
  'people']
  
 ['read' 'one' 'books' 'would' 'time' 'years' 'story' 'reading' 'new'
  'us']
  
 ['like' 'one' 'reading' 'books' 'time' 'story' 'us' 'would' 'years'
  'life']
  
 ['read' 'century' 'one' 'books' 'people' 'reading' 'time' 'would' 'life'
  'story']
  
 ['like' 'books' 'one' 'reading' 'would' 'story' 'life' 'years' 'time'
  'us']
  
 ['read' 'great' 'good' 'would' 'many' 'also' 'well' 'story' 'first'
  'much']
  
 ['one' 'like' 'recommend' 'books' 'years' 'would' 'reading' 'people'
  'life' 'us']
  
 ['like' 'one' 'books' 'recommend' 'reading' 'would' 'time' 'life' 'work'
  'people']
  
 ['one' 'like' 'would' 'reading' 'years' 'books' 'story' 'time' 'people'
  'history']
  
 ['like' 'one' 'would' 'books' 'reading' 'story' 'people' 'time' 'years'
  'way']]